In [2]:
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

In [ ]:
source = 'https://arxiv.org/pdf/2410.23335.pdf'
result = converter.convert(source)

In [4]:
import json

result_dict = result.document.export_to_dict()

print(json.dumps(result_dict, indent=2))


{
  "schema_name": "DoclingDocument",
  "version": "1.8.0",
  "name": "2410.23335v1",
  "origin": {
    "mimetype": "application/pdf",
    "binary_hash": 18351773355608685145,
    "filename": "2410.23335v1.pdf"
  },
  "furniture": {
    "self_ref": "#/furniture",
    "children": [],
    "content_layer": "furniture",
    "name": "_root_",
    "label": "unspecified"
  },
  "body": {
    "self_ref": "#/body",
    "children": [
      {
        "$ref": "#/texts/0"
      },
      {
        "$ref": "#/texts/1"
      },
      {
        "$ref": "#/texts/2"
      },
      {
        "$ref": "#/texts/3"
      },
      {
        "$ref": "#/texts/4"
      },
      {
        "$ref": "#/texts/5"
      },
      {
        "$ref": "#/texts/6"
      },
      {
        "$ref": "#/texts/7"
      },
      {
        "$ref": "#/texts/8"
      },
      {
        "$ref": "#/texts/9"
      },
      {
        "$ref": "#/texts/10"
      },
      {
        "$ref": "#/texts/11"
      },
      {
        "$ref": "#/tex